# Amazon Reviews via Webscraping beschaffen

In diesem Abschnitt werden wir Amazon Kundenbewertungen downloaden und gesammelt in einer Datei speichern. Um den Crawling Prozess zuverlässlicher und schneller zu machen setzen wir einen Proxy Service ein.

**Scraper API - https://www.scraperapi.com/**

Es wird damit auch verhindert dass die Anfrage geblockt wird.

Der Service bietet einen kostenfreie Version bis 5000 Aufrufen im Monat an.

In [297]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urlencode
import csv

Beispielproduktseite

In [298]:
url = 'https://www.amazon.de/SPARSET-Faden-Algenkiller-Schutz-Bewuchs-Schmieralgen/product-reviews/B0036E1MY0/reviewerType=all_reviews/'

### Scraper API 

API Key von Scraper API und Erzeugung einer Anfrage mit der Beispielseite.

In [299]:
params = {'api_key': '8dbc392d7a3b2f0fb3f1fc7f09cc2f77', 'url': url}
response = requests.get('http://api.scraperapi.com/',   params=urlencode(params))

### Beautiful Soup

Beautiful Soup ist eine Python-Bibliothek zum Auslesen von Daten aus HTML- und XML-Dateien. Sie bietet Möglichkeiten zum Navigieren, Suchen und Ändern des Parse-Baums. 

https://www.crummy.com/software/BeautifulSoup/bs4/doc/

Parsen der Anwort mit "Beautiful Soup" und Ausgabe des HTML Codes.

In [300]:
soup = BeautifulSoup(response.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="a-no-js" data-19ax5a9jf="dingo" lang="de-de">
 <!-- sp:feature:head-start -->
 <head>
  <script>
   var aPageStart = (new Date()).getTime();
  </script>
  <meta charset="utf-8"/>
  <!-- sp:end-feature:head-start -->
  <!-- sp:feature:csm:head-open-part1 -->
  <script type="text/javascript">
   var ue_t0=ue_t0||+new Date();
  </script>
  <!-- sp:end-feature:csm:head-open-part1 -->
  <!-- sp:feature:cs-optimization -->
  <meta content="on" http-equiv="x-dns-prefetch-control"/>
  <link href="https://images-eu.ssl-images-amazon.com" rel="dns-prefetch"/>
  <link href="https://m.media-amazon.com" rel="dns-prefetch"/>
  <link href="https://completion.amazon.com" rel="dns-prefetch"/>
  <!-- sp:end-feature:cs-optimization -->
  <!-- sp:feature:csm:head-open-part2 -->
  <script type="text/javascript">
   window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;
if (window.ue_ihb === 1) {

var ue_csm = window,
    ue_hob = +new Date();
(function(d){var e=d.ue=d.ue||{

### Beispiel Extraktion der Usernamen der Bewerter

Suchen des Teilbereichs der Seite der die Reviews enthält. Die entsprechende CSS-Klasse muss im Sourcecode gesucht werden.
Anschließend suchen alle Span-Tags der CSS-Klasse "a-profile-name".

In [301]:
soupreviews = soup.find("div", class_="reviews-content")
soupreviews.find_all("span", class_="a-profile-name")

[<span class="a-profile-name">Norbert Möhler</span>,
 <span class="a-profile-name">Carsten R aus S. </span>,
 <span class="a-profile-name">Paul</span>,
 <span class="a-profile-name">heldenhaft</span>,
 <span class="a-profile-name">Gegenhuber R.</span>,
 <span class="a-profile-name">Jens Gram</span>,
 <span class="a-profile-name">reimmaschine</span>,
 <span class="a-profile-name">Christiane F.</span>,
 <span class="a-profile-name">a.</span>,
 <span class="a-profile-name">elke</span>]

Extraktion und Speicherung der Usernamen in einer Liste

In [302]:
userlist = []
soupreviews = soup.find("div", class_="reviews-content")
for item in soupreviews.find_all("span", class_="a-profile-name"):
    userlist.append(item.get_text())

print(userlist)

['Norbert Möhler', 'Carsten R aus S. ', 'Paul', 'heldenhaft', 'Gegenhuber R.', 'Jens Gram', 'reimmaschine', 'Christiane F.', 'a.', 'elke']


### Beispiel Extraktion der Reviewtexte

In [303]:
soupreviews = soup.find("div", class_="reviews-content")
soupreviews.find_all("span", {"data-hook": "review-body"})

[<span class="a-size-base review-text review-text-content" data-hook="review-body">
 <span>Ich habe drei Wochen nach einbringen in unseren Miniteich gewartet - und war hin und weg.<br/>Das Wasser sauber, die Fische fühlen sich wohl und die Seerosen gedeihen.<br/>Das hat vorher keine Filteranlage geschafft.<br/>DANKE!</span>
 </span>,
 <span class="a-size-base review-text review-text-content" data-hook="review-body">
 <span>Kurz und knapp... es funktioniert! Wir haben einen ca. 2500 Liter Gartenteich und nach ein wenig Geduld (hatte ich am Anfang nicht) sieht man das perfekte Ergebnis.<br/>Um Fragen vorzubeugen... ca. 5-7 Tagen warten!</span>
 </span>,
 <span class="a-size-base review-text review-text-content" data-hook="review-body">
 <span>Die Lieferung ging wie immer schnell, halt Amazon.<br/>Bevor die Entscheidung gefallen ist habe ich mich lange belesen.<br/>Und ich muss sagen dass war genau das richtige. Laut Hersteller beschrieb er das die UV Lampe totaler Käse ist. (Meine Worte)

Extraktion und Speicherung der Texte in einer Liste

In [304]:
reviewlist = []
soupreviews = soup.find("div", class_="reviews-content")
for item in soupreviews.find_all("span", {"data-hook": "review-body"}):
    reviewlist.append(item.get_text())

print(len(reviewlist))

10


In [305]:
print(reviewlist)

['\nIch habe drei Wochen nach einbringen in unseren Miniteich gewartet - und war hin und weg.Das Wasser sauber, die Fische fühlen sich wohl und die Seerosen gedeihen.Das hat vorher keine Filteranlage geschafft.DANKE!\n', '\nKurz und knapp... es funktioniert! Wir haben einen ca. 2500 Liter Gartenteich und nach ein wenig Geduld (hatte ich am Anfang nicht) sieht man das perfekte Ergebnis.Um Fragen vorzubeugen... ca. 5-7 Tagen warten!\n', '\nDie Lieferung ging wie immer schnell, halt Amazon.Bevor die Entscheidung gefallen ist habe ich mich lange belesen.Und ich muss sagen dass war genau das richtige. Laut Hersteller beschrieb er das die UV Lampe totaler Käse ist. (Meine Worte)Und er hat voll Recht. Habe die UV Lampe einige Jahre benutzt. Null Effekt. Wasser grün und Wände immer Verschverschleimt.Wollte schon mein Teich entfernen.Kurz gesagt habe ich mich doch anders entschieden, und habe bei meinen Teich eine große GR gemacht.Ich habe es bis heute in keinster Weise bereut. Dank des Mittels

### Beispiel Extraktion der Bewertungen

In [306]:
soupreviews = soup.find("div", class_="reviews-content")
soupreviews.find_all("span", class_="a-icon-alt")

[<span class="a-icon-alt">5,0 von 5 Sternen</span>,
 <span class="a-icon-alt">5,0 von 5 Sternen</span>,
 <span class="a-icon-alt">5,0 von 5 Sternen</span>,
 <span class="a-icon-alt">5,0 von 5 Sternen</span>,
 <span class="a-icon-alt">5,0 von 5 Sternen</span>,
 <span class="a-icon-alt">5,0 von 5 Sternen</span>,
 <span class="a-icon-alt">5,0 von 5 Sternen</span>,
 <span class="a-icon-alt">3,0 von 5 Sternen</span>,
 <span class="a-icon-alt">5,0 von 5 Sternen</span>,
 <span class="a-icon-alt">5,0 von 5 Sternen</span>]

Extraktion und Speicherung der Texte in einer Liste

In [307]:
starlist = []
soupreviews = soup.find("div", class_="reviews-content")
for item in soupreviews.find_all("span", class_="a-icon-alt"):
    starlist.append(item.get_text().split('von')[0][:-3])

print(starlist)

['5', '5', '5', '5', '5', '5', '5', '3', '5', '5']


### Beispiel Extraktion des Datums

In [308]:
soupreviews = soup.find("div", class_="reviews-content")
soupreviews.find_all("span", {"data-hook": "review-date"})

[<span class="a-size-base a-color-secondary review-date" data-hook="review-date">Rezension aus Deutschland vom 15. Mai 2022</span>,
 <span class="a-size-base a-color-secondary review-date" data-hook="review-date">Rezension aus Deutschland vom 27. April 2022</span>,
 <span class="a-size-base a-color-secondary review-date" data-hook="review-date">Rezension aus Deutschland vom 28. September 2020</span>,
 <span class="a-size-base a-color-secondary review-date" data-hook="review-date">Rezension aus Deutschland vom 20. Juli 2021</span>,
 <span class="a-size-base a-color-secondary review-date" data-hook="review-date">Rezension aus Deutschland vom 17. April 2020</span>,
 <span class="a-size-base a-color-secondary review-date" data-hook="review-date">Rezension aus Deutschland vom 22. Juli 2021</span>,
 <span class="a-size-base a-color-secondary review-date" data-hook="review-date">Rezension aus Deutschland vom 30. Mai 2021</span>,
 <span class="a-size-base a-color-secondary review-date" data-ho

Extraktion und Speicherung der Datumswerte in einer Liste

In [309]:
datelist = []
soupreviews = soup.find("div", class_="reviews-content")
for item in soupreviews.find_all("span", {"data-hook": "review-date"}):
    datelist.append(item.get_text().split('\xa0'))

print(datelist)

[['Rezension aus Deutschland vom 15. Mai 2022'], ['Rezension aus Deutschland vom 27. April 2022'], ['Rezension aus Deutschland vom 28. September 2020'], ['Rezension aus Deutschland vom 20. Juli 2021'], ['Rezension aus Deutschland vom 17. April 2020'], ['Rezension aus Deutschland vom 22. Juli 2021'], ['Rezension aus Deutschland vom 30. Mai 2021'], ['Rezension aus Deutschland vom 7. Juni 2021'], ['Rezension aus Deutschland vom 3. Juni 2021'], ['Rezension aus Deutschland vom 1. Juli 2020']]


### Umblättern

Bisher haben wir immer nur die 10 Einträge der ersten Seite der Bewertungen ausgelesen. Wir möchten aber alle Bewertungen. Dazu muß via Script umgeblättert werden.

Wir uchen den Link zum weiter blättern.

In [310]:
soup.find_all("li", class_="a-last")

[<li class="a-last"><a href="/SPARSET-Faden-Algenkiller-Schutz-Bewuchs-Schmieralgen/product-reviews/B0036E1MY0/ref=cm_cr_arp_d_paging_btm_2?ie=UTF8&amp;pageNumber=2">Weiter<span class="a-letter-space"></span><span class="a-letter-space"></span><span class="larr">→</span></a></li>]

Wir extrahieren den Link zum weiter blättern und bauen den Link zur 2. Seite.

In [311]:
weiter = soup.find("li", class_="a-last")
links = weiter.find("a")
link_url = 'https://www.amazon.de' + links["href"]
print(f"Weiter: {link_url}\n")

Weiter: https://www.amazon.de/SPARSET-Faden-Algenkiller-Schutz-Bewuchs-Schmieralgen/product-reviews/B0036E1MY0/ref=cm_cr_arp_d_paging_btm_2?ie=UTF8&pageNumber=2



### Gesamtes Skript

Wir setzen nun die Teile zusammen und erhalten ein Skript das Bewertungstexte, Rating, Datum und Username holt und jeweils in Listen speichert.

In [312]:
# Erste Seite holen

url = 'https://www.amazon.de/SPARSET-Faden-Algenkiller-Schutz-Bewuchs-Schmieralgen/product-reviews/B0036E1MY0/reviewerType=all_reviews/'
params = {'api_key': '8dbc392d7a3b2f0fb3f1fc7f09cc2f77', 'url': url}
response = requests.get('http://api.scraperapi.com/',   params=urlencode(params))
soup = BeautifulSoup(response.text, 'html.parser')
soupreviews = soup.find("div", class_="reviews-content")
userlist = []
for item in soupreviews.find_all("span", class_="a-profile-name"):
    userlist.append(item.get_text())
reviewlist = []
for item in soupreviews.find_all("span", {"data-hook": "review-body"}):
    reviewlist.append(item.get_text())
starlist = []
for item in soupreviews.find_all("span", class_="a-icon-alt"):
    starlist.append(item.get_text().split('von')[0][:-3])
    datelist = []
for item in soupreviews.find_all("span", {"data-hook": "review-date"}):
    datelist.append(item.get_text().split('\xa0'))

# Weiterblättern Link extrahieren
weiter = soup.find("li", class_="a-last")
links = weiter.find("a")
if links is not None:
    link_url = 'https://www.amazon.de' + links["href"]
else:
    link_url = ''

# Weitere Seiten holen so lange ein weiter-Link verfügbar ist
while link_url != '':
    params = {'api_key': '8dbc392d7a3b2f0fb3f1fc7f09cc2f77', 'url': link_url}
    response = requests.get('http://api.scraperapi.com/',   params=urlencode(params))
    soup = BeautifulSoup(response.text, 'html.parser')
    soupreviews = soup.find("div", class_="reviews-content")
    for item in soupreviews.find_all("span", class_="a-profile-name"):
        userlist.append(item.get_text())
    for item in soupreviews.find_all("span", {"data-hook": "review-body"}):
        reviewlist.append(item.get_text())
    for item in soupreviews.find_all("span", class_="a-icon-alt"):
        starlist.append(item.get_text().split('von')[0][:-3])
    for item in soupreviews.find_all("span", {"data-hook": "review-date"}):
        datelist.append(item.get_text().split('\xa0'))

    weiter = soup.find("li", class_="a-last")
    links = weiter.find("a")
    if links is not None:
        link_url = 'https://www.amazon.de' + links["href"]
    else:
        link_url = ''

print(userlist)

['Norbert Möhler', 'Carsten R aus S. ', 'Paul', 'heldenhaft', 'Gegenhuber R.', 'Jens Gram', 'reimmaschine', 'Christiane F.', 'a.', 'elke', 'Dieter', 'Peter Hofschweiger', 'Amazon Kunde', 'Florian B.', 'Amazon Kunde', 'Amazon Kunde', 'Sabine Reinke', 'Svenja S.', 'Ich', 'Kmetic A.', 'Helmut Lindner', 'Jendrik Nickel', 'Gloria', 'Blue', 'Martin B.', 'Isabelle G.', 'Betty', 'Marion Brunold', 'Birgit', 'hanso', 'Hundertmark', 'Dieter Reith', 'MO', 'Schnucki']


Zusammenfügen aller Listen zu einem Dataframe.

In [313]:
df = pd.DataFrame(list(zip(userlist, datelist, starlist, reviewlist)),
               columns =['Name', 'Datum', 'Sterne', 'Reviews'])
df.head()

,Name,Datum,Sterne,Reviews
0,Norbert Möhler,[Rezension aus Deutschland vom 15. Mai 2022],5,\nIch habe drei Wochen nach einbringen in unse...
1,Carsten R aus S.,[Rezension aus Deutschland vom 27. April 2022],5,\nKurz und knapp... es funktioniert! Wir haben...
2,Paul,[Rezension aus Deutschland vom 28. September 2...,5,"\nDie Lieferung ging wie immer schnell, halt A..."
3,heldenhaft,[Rezension aus Deutschland vom 20. Juli 2021],5,\nDie Lieferung ging zügig von statten. Das Sp...
4,Gegenhuber R.,[Rezension aus Deutschland vom 17. April 2020],5,\nIch habe mich heute sehr gewundert als plötz...


### Speichern des Dataframes

In [314]:
df.to_csv('amazon.csv', index=False)

### Wieder einlesen des Dataframes

In [ ]:
newdf = pd.read_csv('amazon.csv')  
newdf.info()

## Weitere Produktinformationen auslesen

In [316]:
url2 = 'https://www.amazon.de/SPARSET-Faden-Algenkiller-Schutz-Bewuchs-Schmieralgen/dp/B0036E1MY0/'
params = {'api_key': '8dbc392d7a3b2f0fb3f1fc7f09cc2f77', 'url': url2}
response = requests.get('http://api.scraperapi.com/',   params=urlencode(params))
soup = BeautifulSoup(response.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="a-no-js" data-19ax5a9jf="dingo" lang="de-de">
 <!-- sp:feature:head-start -->
 <head>
  <script>
   var aPageStart = (new Date()).getTime();
  </script>
  <meta charset="utf-8"/>
  <!-- sp:end-feature:head-start -->
  <!-- sp:feature:csm:head-open-part1 -->
  <script type="text/javascript">
   var ue_t0=ue_t0||+new Date();
  </script>
  <!-- sp:end-feature:csm:head-open-part1 -->
  <!-- sp:feature:cs-optimization -->
  <meta content="on" http-equiv="x-dns-prefetch-control"/>
  <link href="https://images-eu.ssl-images-amazon.com" rel="dns-prefetch"/>
  <link href="https://m.media-amazon.com" rel="dns-prefetch"/>
  <link href="https://completion.amazon.com" rel="dns-prefetch"/>
  <!-- sp:end-feature:cs-optimization -->
  <!-- sp:feature:csm:head-open-part2 -->
  <script type="text/javascript">
   window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;
if (window.ue_ihb === 1) {

var ue_csm = window,
    ue_hob = +new Date();
(function(d){var e=d.ue=d.ue||{

### Produkttitel

In [317]:
titeltext = soup.find("span", id="productTitle")
titeltext  = titeltext.get_text()
titeltext

'        Weitz-Wasserwelt Algen im Teich! - TOP TEICHPFLEGE IM SPARSET (Sparset Faden-Algenkiller)       '

### Produktfeatures

In [318]:
features = soup.find("div", id="feature-bullets")
features  = features.get_text()
features

'   Schutz vor Bewuchs von Faden- und Schmieralgen    Hochwirksam gegen Faden- und Schmieralgen    Optimaler Oberflächenschutz im Teich    Erhöht den Sauerstoffgehalt    Tausendfach bewährt    \n'

### Produktbeschreibung

In [319]:
beschreibung = soup.find("div", id="productDescription_feature_div")
beschreibung  = beschreibung.get_text()
beschreibung

'\n\n   Produktbeschreibungen    \n  Algenkiller Protect 150 g für bis zu 10.000 Liter. Das Erfolgsprodukt - Die absolute Nr. 1 schützt Oberflächen im Teich vor Bewuchs von Faden- und Schmieralgen. Faden-Algenkiller 500 g für bis zu 10.000 Liter. Für eine einfache, gezielte und erfolgreiche Beseitigung von Fadenalgen. Starter-Bakterien 150 g für bis zu 10.000 Liter. Hochaktive Mikroorganismen bewirken biologisch gesunde Garten- und Schwimmteiche. Durch die biobird Starterbakterien wird ein schnelles Anfahren eines jeden Teiches zu einem funktionsfähigen Biosystem ermöglicht.Inhalts- und Wirkstoffe Algenkiller Protect: Lebende, nicht genmanipulierte Mikroorganismen (6x10h9), Silizium, Zeolith, Calcium, Magnesium, etc. Terbutryn (2 mg/g), Monolinuron (3 mg/g)Gebrauchsanweisung Faden-Algenkiller: Bestmöglich einfach auf die Fadenalgen aufstreuen. Das Granulat wirkt sofort bei gezielter Anwendung.Gebrauchsanweisung zu biobird Starter-Bakterien: 150g Starterbakterien pro 10.000 ltr. Teichwa

## Infos zu Produkt extrahieren

Eine Seite der Reviews holen

In [320]:
def extract_one_review(url, userlist, reviewlist, starlist, datelist):
    params = {'api_key': '8dbc392d7a3b2f0fb3f1fc7f09cc2f77', 'url': url}
    response = requests.get('http://api.scraperapi.com/',   params=urlencode(params))
    soup = BeautifulSoup(response.text, 'html.parser')
    soupreviews = soup.find("div", class_="reviews-content")
    
    if soupreviews != None:
        reviewblock = soupreviews.find_all("div", {"data-hook": "review"})
        for item in reviewblock:
            date = item.find("span", {"data-hook": "review-date"})
            if item != None and date.get_text().startswith('Rezension aus Deutschland'):
                user = item.find("span", class_="a-profile-name")
                if user != None:
                    userlist.append(user.get_text())
                else:
                    userlist.append('')
                body = item.find("span", {"data-hook": "review-body"})
                if body != None:
                    reviewlist.append(body.get_text())
                else:
                    reviewlist.append('')
                star = item.find("span", class_="a-icon-alt")
                if star != None:
                    starlist.append(star.get_text().split('von')[0][:-3])
                else:
                    starlist.append('')
                date = item.find("span", {"data-hook": "review-date"})
                if date != None:
                    datelist.append(date.get_text().split('vom')[1][1:])
                else:
                    datelist.append('')
                    

    weiter = soup.find("li", class_="a-last")
    if weiter != None:
        links = weiter.find("a")
    else:
        links = None
    if links is not None:
        link_url = 'https://www.amazon.de' + links["href"]
    else:
        link_url = ''
    return {'user': userlist, 'reviews': reviewlist, 'bewertungen': starlist, 'reviewdates': datelist, 'nexturl': link_url}

In [321]:
userlist = []
reviewlist = []
starlist = []
datelist = []

url = 'https://www.amazon.de/SPARSET-Faden-Algenkiller-Schutz-Bewuchs-Schmieralgen/product-reviews/B0036E1MY0/'
extract_one_review(url, userlist, reviewlist, starlist, datelist)

{'user': [],
 'reviews': [],
 'bewertungen': [],
 'reviewdates': [],
 'nexturl': 'https://www.amazon.de/SPARSET-Faden-Algenkiller-Schutz-Bewuchs-Schmieralgen/product-reviews/B0036E1MY0?pageNumber=2'}

Alle Reviews holen mit umblättern.

In [322]:
def extract_reviews(url):
    userlist = []
    reviewlist = []
    starlist = []
    datelist = []
    
    response = extract_one_review(url, userlist, reviewlist, starlist, datelist)
    link_url = response["nexturl"]
    
    while link_url != '':
        response = extract_one_review(link_url, list(response["user"]), list(response["reviews"]), list(response["bewertungen"]), 
                                      list(response["reviewdates"]))
        link_url = response["nexturl"]
    
    return {'user': response['user'], 'reviews': response['reviews'], 'bewertungen': response['bewertungen'], 
            'reviewdates': response['reviewdates']}
    


In [323]:
url = 'https://www.amazon.de/SPARSET-Faden-Algenkiller-Schutz-Bewuchs-Schmieralgen/product-reviews/B0036E1MY0/'
extract_reviews(url)

{'user': ['Norbert Möhler',
  'Carsten R aus S. ',
  'Paul',
  'heldenhaft',
  'Gegenhuber R.',
  'Jens Gram',
  'reimmaschine',
  'Christiane F.',
  'a.',
  'elke',
  'Dieter',
  'Peter Hofschweiger',
  'Amazon Kunde',
  'Florian B.',
  'Amazon Kunde',
  'Amazon Kunde',
  'Sabine Reinke',
  'Svenja S.',
  'Ich',
  'Kmetic A.'],
 'reviews': ['\nIch habe drei Wochen nach einbringen in unseren Miniteich gewartet - und war hin und weg.Das Wasser sauber, die Fische fühlen sich wohl und die Seerosen gedeihen.Das hat vorher keine Filteranlage geschafft.DANKE!\n',
  '\nKurz und knapp... es funktioniert! Wir haben einen ca. 2500 Liter Gartenteich und nach ein wenig Geduld (hatte ich am Anfang nicht) sieht man das perfekte Ergebnis.Um Fragen vorzubeugen... ca. 5-7 Tagen warten!\n',
  '\nDie Lieferung ging wie immer schnell, halt Amazon.Bevor die Entscheidung gefallen ist habe ich mich lange belesen.Und ich muss sagen dass war genau das richtige. Laut Hersteller beschrieb er das die UV Lampe tot

Funktion für die Produktdetails.

In [324]:
def extract_details(url):
    params = {'api_key': '8dbc392d7a3b2f0fb3f1fc7f09cc2f77', 'url': url}
    response = requests.get('http://api.scraperapi.com/',   params=urlencode(params))
    soup = BeautifulSoup(response.text, 'html.parser')

    titeltext = soup.find("span", id="productTitle")
    if titeltext != None:
        titeltext = titeltext.get_text()
    else:
        titeltext = ''
    
    features = soup.find("div", id="feature-bullets")
    if features != None:
        features  = features.get_text()
    else:
        features = ''
  
    beschreibung = soup.find("div", id="productDescription_feature_div")
    if beschreibung != None:
        beschreibung  = beschreibung.get_text()
    else:
        beschreibung = ''
    
    reviewurl = soup.find("a", {"data-hook": "see-all-reviews-link-foot"})
    
    if reviewurl != None:
        reviewurl = 'https://www.amazon.de' + reviewurl["href"]
    
    response = extract_reviews(reviewurl)
    
    return {'titel': titeltext, 'features': features, 'beschreibung': beschreibung, 'user': response['user'], 
            'reviews': response['reviews'], 'bewertungen': response['bewertungen'], 'reviewdates': response['reviewdates']}


In [325]:
url = 'https://www.amazon.de/SPARSET-Faden-Algenkiller-Schutz-Bewuchs-Schmieralgen/dp/B0036E1MY0/'
extract_details(url)

{'titel': '        Weitz-Wasserwelt Algen im Teich! - TOP TEICHPFLEGE IM SPARSET (Sparset Faden-Algenkiller)       ',
 'features': '     Schutz vor Bewuchs von Faden- und Schmieralgen   Hochwirksam gegen Faden- und Schmieralgen   Optimaler Oberflächenschutz im Teich   Erhöht den Sauerstoffgehalt   Tausendfach bewährt   \n Mehr Produktdetails anzeigen   ',
 'beschreibung': '\n\n   Produktbeschreibungen    \n  Algenkiller Protect 150 g für bis zu 10.000 Liter. Das Erfolgsprodukt - Die absolute Nr. 1 schützt Oberflächen im Teich vor Bewuchs von Faden- und Schmieralgen. Faden-Algenkiller 500 g für bis zu 10.000 Liter. Für eine einfache, gezielte und erfolgreiche Beseitigung von Fadenalgen. Starter-Bakterien 150 g für bis zu 10.000 Liter. Hochaktive Mikroorganismen bewirken biologisch gesunde Garten- und Schwimmteiche. Durch die biobird Starterbakterien wird ein schnelles Anfahren eines jeden Teiches zu einem funktionsfähigen Biosystem ermöglicht.Inhalts- und Wirkstoffe Algenkiller Protect:

In [326]:
datadict = extract_details(url)
df = pd.DataFrame.from_dict(datadict)
df.head()

,titel,features,beschreibung,user,reviews,bewertungen,reviewdates
0,Weitz-Wasserwelt Algen im Teich! - TOP...,Schutz vor Bewuchs von Faden- und Schmier...,,Dieter,\nDas Mittel beziehen wir schon viele Jahre. U...,5,10. April 2021
1,Weitz-Wasserwelt Algen im Teich! - TOP...,Schutz vor Bewuchs von Faden- und Schmier...,,Peter Hofschweiger,\nAlles bestens !!\n,5,18. Juni 2021
2,Weitz-Wasserwelt Algen im Teich! - TOP...,Schutz vor Bewuchs von Faden- und Schmier...,,Amazon Kunde,\nWir haben die beiden Mittel gegen Grünalgenb...,5,10. September 2018
3,Weitz-Wasserwelt Algen im Teich! - TOP...,Schutz vor Bewuchs von Faden- und Schmier...,,Florian B.,"\nHat super geklappt, Teich sieht toll aus.\n",5,22. April 2021
4,Weitz-Wasserwelt Algen im Teich! - TOP...,Schutz vor Bewuchs von Faden- und Schmier...,,Amazon Kunde,\nDas Produkt kaufen wir jedes Jahr wieder\n,5,9. März 2021


Speichern

In [327]:
df.to_csv('amazon_suche.csv', index=False)

In [328]:
df.tail()

,titel,features,beschreibung,user,reviews,bewertungen,reviewdates
19,Weitz-Wasserwelt Algen im Teich! - TOP...,Schutz vor Bewuchs von Faden- und Schmier...,,hanso,"\nAusgangslage: Teich etwa 15 cbm groß, Halbsc...",1,4. Oktober 2014
20,Weitz-Wasserwelt Algen im Teich! - TOP...,Schutz vor Bewuchs von Faden- und Schmier...,,Hundertmark,\nMein Teich war grün und voller Fadenalgen . ...,5,10. Juli 2022
21,Weitz-Wasserwelt Algen im Teich! - TOP...,Schutz vor Bewuchs von Faden- und Schmier...,,Dieter Reith,\nSuper\n,5,22. März 2021
22,Weitz-Wasserwelt Algen im Teich! - TOP...,Schutz vor Bewuchs von Faden- und Schmier...,,MO,"\nBenutze den Algenkiller seit Jahren, habe na...",5,23. März 2022
23,Weitz-Wasserwelt Algen im Teich! - TOP...,Schutz vor Bewuchs von Faden- und Schmier...,,Schnucki,"\nWir kaufen schon viele Jahre diese Produkte,...",5,19. März 2021
